<a target="_blank" href="../cluster" style="font-size:20px">All Applications (YARN)</a>

# HDFS configuration

https://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-hdfs/hdfs-default.xml

In [1]:
! cat $HADOOP_HOME/etc/hadoop/hdfs-site.xml

<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>
	<property>
		<name>dfs.blocksize</name>
		<value>16m</value>
	</property>
	<property>
		<name>dfs.replication</name>
		<value>1</value>
	</property>
	<property>
		<name>dfs.permissions.enabled</name>
		<value>false</value>
	</property>
	<property>
		<name>dfs.namenode.name.dir</name>
		<value>file:///usr/local/hadoop/hdfs/namenode</value>
	</property>
	<property>
		<name>dfs.datanode.data.dir</name>
		<value>file:///usr/local/hadoop/hdfs/datanode</value>
	</property>
</configuration>


In [2]:
! ls /usr/local/hadoop/hdfs

datanode  namenode


# Available commands

In [3]:
! hadoop fs

Usage: hadoop fs [generic options]
	[-appendToFile <localsrc> ... <dst>]
	[-cat [-ignoreCrc] <src> ...]
	[-checksum [-v] <src> ...]
	[-chgrp [-R] GROUP PATH...]
	[-chmod [-R] <MODE[,MODE]... | OCTALMODE> PATH...]
	[-chown [-R] [OWNER][:[GROUP]] PATH...]
	[-concat <target path> <src path> <src path> ...]
	[-copyFromLocal [-f] [-p] [-l] [-d] [-t <thread count>] [-q <thread pool queue size>] <localsrc> ... <dst>]
	[-copyToLocal [-f] [-p] [-crc] [-ignoreCrc] [-t <thread count>] [-q <thread pool queue size>] <src> ... <localdst>]
	[-count [-q] [-h] [-v] [-t [<storage type>]] [-u] [-x] [-e] [-s] <path> ...]
	[-cp [-f] [-p | -p[topax]] [-d] [-t <thread count>] [-q <thread pool queue size>] <src> ... <dst>]
	[-createSnapshot <snapshotDir> [<snapshotName>]]
	[-deleteSnapshot <snapshotDir> <snapshotName>]
	[-df [-h] [<path> ...]]
	[-du [-s] [-h] [-v] [-x] <path> ...]
	[-expunge [-immediate] [-fs <path>]]
	[-find <path> ... <expression> ...]
	[-get [-f] [-p] [-crc] [-ignoreCrc] [-t <thread count>

# Copy files to/from HDFS

In [4]:
! echo "Test file" > test.txt

In [5]:
! ls -l .

total 168504
drwxr-sr-x 3 jovyan users     4096 Apr  9 08:19 cpu1
-rw-rw-r-- 1 jovyan root     21898 Apr  7 16:24 hdfs-basics-seminar.ipynb
-rw-r--r-- 1 jovyan users      446 Apr  8 19:23 mapper2.py
-rw-r--r-- 1 jovyan users      601 Apr  8 19:48 mapper3.py
-rw-r--r-- 1 jovyan users      287 Apr  8 18:36 mapper.py
-rw-r--r-- 1 jovyan users 57408674 Apr 11 19:43 mapreduce-hw-task-Copy1.ipynb
-rw-r--r-- 1 jovyan users 57408674 Apr 11 19:43 mapreduce-hw-task-Copy2.ipynb
-rw-rw-r-- 1 jovyan root  57408674 Apr 10 13:01 mapreduce-hw-task.ipynb
-rw-rw-r-- 1 jovyan root    244931 Apr  8 19:54 mapreduce-wordcount-seminar.ipynb
-rw-r--r-- 1 jovyan users      164 Apr  8 19:27 reducer2.py
-rw-r--r-- 1 jovyan users      381 Apr  8 18:37 reducer.py
-rw-r--r-- 1 jovyan users       57 Apr  9 10:51 result.json
-rw-r--r-- 1 jovyan users        5 Apr  8 19:27 result.txt
-rw-r--r-- 1 jovyan users       10 Apr  7 13:31 test2.txt
-rw-r--r-- 1 jovyan users       10 Apr 12 21:01 test.txt
drwxr-sr-x 1 jovyan r

In [6]:
! hadoop fs -copyFromLocal test.txt /

copyFromLocal: `/test.txt': File exists


In [7]:
! hadoop fs -ls /

Found 8 items
drwxr-xr-x   - jovyan supergroup          0 2023-04-08 18:25 /test
-rw-r--r--   1 jovyan supergroup         10 2023-04-03 18:53 /test.txt
-rw-r--r--   1 jovyan supergroup          8 2023-04-07 13:31 /test3.txt
drwxrwx---   - root   supergroup          0 2023-04-08 18:20 /tmp
drwxr-xr-x   - jovyan supergroup          0 2023-04-09 09:24 /user
drwxr-xr-x   - jovyan supergroup          0 2023-04-08 18:19 /wiki
drwxr-xr-x   - jovyan supergroup          0 2023-04-08 18:23 /word-count
drwxr-xr-x   - jovyan supergroup          0 2023-04-06 14:35 /yandex_music


In [8]:
! hadoop fs -copyToLocal /test.txt test2.txt

copyToLocal: `test2.txt': File exists


In [9]:
! cat test2.txt

Test file


# Streaming from/to HDFS

In [10]:
! hadoop fs -cat /test.txt | wc -w

2


In [11]:
! echo "1 2 3 4" | hadoop fs -copyFromLocal - /test3.txt

copyFromLocal: `/test3.txt': File exists


In [12]:
! hadoop fs -cat /test3.txt

1 2 3 4


# Change the replication factor

In [13]:
! cat $HADOOP_HOME/etc/hadoop/hdfs-site.xml | grep -A 1 replication 

		<name>dfs.replication</name>
		<value>1</value>


In [14]:
! hadoop fs -stat %r /test.txt

1


In [15]:
! hadoop fs -setrep 2 /test.txt

Replication 2 set: /test.txt


In [16]:
! hadoop fs -stat %r /test.txt

2


# HDFS block size

In [17]:
! cat $HADOOP_HOME/etc/hadoop/hdfs-site.xml | grep -A 1 block 

		<name>dfs.blocksize</name>
		<value>16m</value>


In [18]:
%%file generate.py
import sys

megabytes = int(sys.argv[1])
string = "{}M\n".format(megabytes)
count = 1024 * 1024 * megabytes // len(string) - 1
print(string * count)

Writing generate.py


In [19]:
! python ./generate.py 1 | head -n 5

1M
1M
1M
1M
1M
Traceback (most recent call last):
  File "./generate.py", line 6, in <module>
    print(string * count)
BrokenPipeError: [Errno 32] Broken pipe


In [20]:
! hadoop fs -rm /16.txt /32.txt

rm: `/16.txt': No such file or directory
rm: `/32.txt': No such file or directory


In [21]:
# 16 MB file
! python ./generate.py 16 | hadoop fs -copyFromLocal - /16.txt

# 32 MB file
! python ./generate.py 32 | hadoop fs -copyFromLocal - /32.txt

In [22]:
! hdfs fsck / -files -blocks

Connecting to namenode via http://localhost:9870/fsck?ugi=jovyan&files=1&blocks=1&path=%2F
FSCK started by jovyan (auth:SIMPLE) from /127.0.0.1 for path / at Tue Oct 27 18:01:09 GMT 2020

/ <dir>
/16.txt 16777213 bytes, replicated: replication=1, 1 block(s):  OK
0. BP-417124827-172.18.0.2-1603454947911:blk_1073741827_1003 len=16777213 Live_repl=1

/32.txt 33554429 bytes, replicated: replication=1, 2 block(s):  OK
0. BP-417124827-172.18.0.2-1603454947911:blk_1073741828_1004 len=16777216 Live_repl=1
1. BP-417124827-172.18.0.2-1603454947911:blk_1073741829_1005 len=16777213 Live_repl=1

/test.txt 10 bytes, replicated: replication=2, 1 block(s):  Under replicated BP-417124827-172.18.0.2-1603454947911:blk_1073741825_1001. Target Replicas is 2 but found 1 live replica(s), 0 decommissioned replica(s), 0 decommissioning replica(s).
0. BP-417124827-172.18.0.2-1603454947911:blk_1073741825_1001 len=10 Live_repl=1

/test3.txt 8 bytes, replicated: replication=1, 1 block(s):  OK
0. BP-417124827-172.1

# What HDFS looks like on a disk

In [23]:
! cat $HADOOP_HOME/etc/hadoop/hdfs-site.xml | grep -A 1 dir

		<name>dfs.namenode.name.dir</name>
		<value>file:///usr/local/hadoop/hdfs/namenode</value>
--
		<name>dfs.datanode.data.dir</name>
		<value>file:///usr/local/hadoop/hdfs/datanode</value>


In [24]:
! ls -R $HADOOP_HOME/hdfs/datanode

/usr/local/hadoop/hdfs/datanode:
current  in_use.lock

/usr/local/hadoop/hdfs/datanode/current:
BP-417124827-172.18.0.2-1603454947911  VERSION

/usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911:
current  scanner.cursor  tmp

/usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current:
finalized  rbw	VERSION

/usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/finalized:
subdir0

/usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/finalized/subdir0:
subdir0

/usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/finalized/subdir0/subdir0:
blk_1073741825		  blk_1073741827	    blk_1073741829
blk_1073741825_1001.meta  blk_1073741827_1003.meta  blk_1073741829_1005.meta
blk_1073741826		  blk_1073741828
blk_1073741826_1002.meta  blk_1073741828_1004.meta

/usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/rbw

In [25]:
! find $HADOOP_HOME/hdfs/datanode | grep blk | xargs head -c 32

==> /usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/finalized/subdir0/subdir0/blk_1073741827 <==
16M
16M
16M
16M
16M
16M
16M
16M

==> /usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/finalized/subdir0/subdir0/blk_1073741826_1002.meta <==
    ,��
==> /usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/finalized/subdir0/subdir0/blk_1073741829 <==
32M
32M
32M
32M
32M
32M
32M
32M

==> /usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/finalized/subdir0/subdir0/blk_1073741829_1005.meta <==
    �FmG�FmG�FmG�FmG�FmG�FmG�
==> /usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/finalized/subdir0/subdir0/blk_1073741828 <==
32M
32M
32M
32M
32M
32M
32M
32M

==> /usr/local/hadoop/hdfs/datanode/current/BP-417124827-172.18.0.2-1603454947911/current/finalized/subdir0/subdir0/blk_1073741825_1001.meta <==
    [H�
==> /

In [26]:
! ls -R $HADOOP_HOME/hdfs/namenode

/usr/local/hadoop/hdfs/namenode:
current  in_use.lock

/usr/local/hadoop/hdfs/namenode/current:
edits_0000000000000000001-0000000000000000009  fsimage_0000000000000000000.md5
edits_inprogress_0000000000000000010	       seen_txid
fsimage_0000000000000000000		       VERSION
